# Temperaturstatistiken für Europa

Dieser Datensatz enthält Temperaturstatistiken für Europa, die sich auf die tägliche 2-Meter-Lufttemperatur (Mittelwert, Minimum und Maximum) für ganze Jahre sowie für die Winter- (DJF: Dezember-Januar-Februar) und die Sommersaison (JJA: Juni-Juli-August) konzentrieren. Die Statistiken, die aus den um Verzerrungen bereinigten EURO-CORDEX-Daten abgeleitet wurden, umfassen einen geglätteten 30-Jahres-Durchschnitt von 1971 bis 2100, was zu einer Zeitreihe von 1986 bis 2085 führt, einschließlich der Mittelwerte des Modellensembles und der zugehörigen Standardabweichungen. Diese Temperaturperzentile sind im öffentlichen Gesundheitswesen und in der Epidemiologie von entscheidender Bedeutung für die Bewertung von Gesundheitsrisiken und -auswirkungen und ermöglichen Vergleiche zwischen verschiedenen Regionen unter verschiedenen Klimawandelszenarien.

**Schnellnavigation:**
* [Herunterladen und Entpacken des Datensatzes](#herunterladen-und-entpacken-des-datensatzes)
* [Untersuchen der Metadaten der netCDF4-Datei](#untersuchen-der-metadaten-der-netcdf4-datei)
* [Exportieren der NetCDF4-Dateien im csv-Format](#exportieren-der-netcdf4-dateien-im-csv-format)
* [Exportieren der NetCDF4-Datei nach GeoTIFF](#export-des-netcdf4-datei-nach-geotiff)
* [Analyse und Visualisierung Optionen](#analyse-und-visualisierung-optionen)

**Information on Dataset:**
* Quelle: [Temperature Statistics for Europe](https://cds.climate.copernicus.eu/datasets/sis-temperature-statistics?tab=overview)
* Author: T. Tewes (Stadt Konstanz) 
* Notebook Version: 1.1 (Updated: December 17. 2024)

## 1. Festlegen der Pfade und Arbeitsverzeichnisse

In [23]:
import os

''' ---- Verzeichnisse hier angeben ---- '''
download_folder = r".\data\sis-temperature-statistics\download"
working_folder = r".\data\sis-temperature-statistics\working"
geotiff_folder = r".\data\sis-temperature-statistics\geotiff"
csv_folder = r".\data\sis-temperature-statistics\csv"
output_folder = r".\data\sis-temperature-statistics\output"
''' ----- Ende der Angaben ---- '''

os.makedirs(download_folder, exist_ok=True)
os.makedirs(working_folder, exist_ok=True)
os.makedirs(geotiff_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

## 2. Herunterladen und Entpacken des Datensatzes

### 2.1 Authentifizierung

In [24]:
import cdsapi

def main():
    # API-Key für die Authentifizierung
    api_key = "fdae60fd-35d4-436f-825c-c63fedab94a4"
    api_url = "https://cds.climate.copernicus.eu/api"

    # Erstellung des CDS-API-Clients
    client = cdsapi.Client(url=api_url, key=api_key)
    return client

### 2.2 Definieren Sie die „request“ und laden Sie den Datensatz herunter

Definieren Sie zusätzliche Anfragefelder, um sicherzustellen, dass die Anfrage innerhalb der Dateigrößenbeschränkung bleibt. Bei der Arbeit mit Geodaten oder APIs, die Karten- oder Satellitenbilder zurückgeben, kann die Begrenzung des geografischen Interessengebiets verhindern, dass Anfragen zu groß werden und die Datei- oder Verarbeitungsgrenzen überschreiten. Begrenzungsrahmen (Bounding Boxes) werden verwendet, um das geografische Gebiet für solche Anfragen festzulegen.

Die untenstehenden Koordinaten wurden mit dem Tool <a href="https://str-ucture.github.io/bbox-extractor/" target="_blank">BBox Extractor</a> ermittelt.

*BBox Extractor ist ein webbasiertes Tool, das Benutzern hilft, interaktiv Begrenzungsrahmen-Koordinaten im WGS84-Format (Breite/Länge) auszuwählen und zu generieren. Dies ist besonders nützlich für APIs oder Datensätze, die eine Eingabe eines geografischen Gebiets erfordern*

In [25]:
# Definieren der Begrenzungsrahmen-Koordinaten (WGS84-Format)
# Das Koordinatenformat lautet: [Norden, Westen, Süden, Osten]
bbox_wgs84_deutschland = [56.0, 5.8, 47.2, 15.0]
bbox_wgs84_konstanz = [47.9, 8.9, 47.6, 9.3]

In [26]:
# Alternativ können Sie ein Shapefile für eine präzise geografische Filterung verwenden
import geopandas as gpd
import math

# Beispiel: Shapefile von Konstanz laden (WGS84-Projektion)
de_shapefile = r"./shapefiles/de_boundary.shp"
de_gdf = gpd.read_file(de_shapefile)

# Extrahieren Sie den Begrenzungsrahmen des Shapefiles
de_bounds = de_gdf.total_bounds

# Passen Sie den Begrenzungsrahmen an und puffern Sie ihn, um einen etwas größeren
de_bounds_adjusted = [(math.floor(de_bounds[0]* 10)/10)-0.1,
                      (math.floor(de_bounds[1]* 10)/10)-0.1,
                      (math.ceil(de_bounds[2]* 10)/10)+0.1,
                      (math.ceil(de_bounds[3]* 10)/10)+0.1]

# Ordnen Sie die Koordinaten in das Format: [Nord, West, Süd, Ost] um.
bbox_de_bounds_adjusted = [de_bounds_adjusted[3], de_bounds_adjusted[0],
                           de_bounds_adjusted[1], de_bounds_adjusted[2]]

In [27]:
# Definieren Sie verfügbare Optionen für Zeiträume und Temperaturvariablen
period_list = ["year", "summer", "winter"]
variable_list = ["average_temperature", "minimum_temperature", "maximum_temperature"]

# Wählen Sie den Zeitraum und die Variable für die Verarbeitung aus; ändern Sie diese nach Bedarf
# Standardmäßig auf "Jahr" und "Durchschnittstemperatur" eingestellt
selected_period = period_list[0]  # Standard: "year"
selected_variable = variable_list[0]  # Standard: "average_temperature"

# Ausgewählte Variable und Periode anzeigen
print(f"Ausgewählte Variable: {selected_variable}\nAusgewählter Zeitraum: {selected_period}")

# Definieren Sie Statistiken basierend auf der ausgewählten Variable
# Fügen Sie für "average_temperature" zusätzliche Statistiken ein; verwenden Sie andernfalls nur "time_average"
statistic = (
    ['time_average', '10th_percentile', '90th_percentile']
    if selected_variable == "average_temperature"
    else ['time_average']
)

Ausgewählte Variable: average_temperature
Ausgewählter Zeitraum: year


In [28]:
# Definition des Datensatzes und der Request-Parameter
dataset = "sis-temperature-statistics"
request = {
    "variable": selected_variable,
    "period": selected_period,
    "statistic": statistic,
    "experiment": [
        "rcp4_5",
        "rcp8_5"
    ],
    "ensemble_statistic": [
        "ensemble_members_average",
        "ensemble_members_standard_deviation"
    ],
    "area": bbox_de_bounds_adjusted
}

In [29]:
# Führen Sie es aus, um den Datensatz herunterzuladen:
def main_retrieve():
    dataset_filename = f"{dataset}_{selected_period}_{selected_variable}.zip"
    dataset_filepath = os.path.join(download_folder, dataset_filename)
    
    # Den Datensatz nur herunterladen, wenn er noch nicht heruntergeladen wurde
    if not os.path.isfile(dataset_filepath):
        # Rufen Sie den CDS-Client nur auf, wenn der Datensatz noch nicht heruntergeladen wurde.
        client = main()
        # Den Datensatz mit den definierten Anforderungsparametern herunterladen
        client.retrieve(dataset, request, dataset_filepath)
    else:
        print("Datensatz bereits heruntergeladen.")
        
if __name__ == "__main__":
    main_retrieve()

Datensatz bereits heruntergeladen.


### 2.3 Extrahieren Sie die ZIP-Dateien in Ordner

In [30]:
import zipfile

# Definieren Sie den Ordner, in dem die extrahierten Dateien gespeichert werden
extract_folder = os.path.join(working_folder, selected_period)
os.makedirs(extract_folder, exist_ok=True)

# Entpacken der ZIP-Datei
try:
    if not os.listdir(extract_folder):
        for variable in variable_list:
            # Konstruieren Sie den Dateinamen für die ZIP-Datei basierend auf dem ausgewählten Zeitraum und der Variable
            dataset_filename = f"{dataset}_{selected_period}_{variable}.zip"
            dataset_filepath = os.path.join(download_folder, dataset_filename)
            
            # Versuchen Sie, die ZIP-Datei zu öffnen und zu extrahieren
            with zipfile.ZipFile(dataset_filepath, 'r') as zip_ref:
                zip_ref.extractall(extract_folder)
                print(f"Dateien erfolgreich extrahiert nach: {extract_folder}")
    else:
        print("Ordner ist nicht leer. Entpacken überspringen.")
except FileNotFoundError:
    print(f"Fehler: Die Datei {dataset_filepath} wurde nicht gefunden.")
except zipfile.BadZipFile:
    print(f"Fehler: Die Datei {dataset_filepath} ist keine gültige ZIP-Datei.")
except Exception as e:
    print(f"Ein unerwarteter Fehler ist aufgetreten: {e}")

Ordner ist nicht leer. Entpacken überspringen.


## 3. Untersuchen die Metadaten der NetCDF4-Datei

⚠️ <span style="background-color: red; color: white; padding: 2px 4px;">Wichtig</span> Nachdem Sie die Datensätze heruntergeladen haben, **aktualisieren Sie** `selected_period` im Codeblock unten, um mit der Analyse, Datenextraktion und Visualisierung fortzufahren. Dieser Schritt ist entscheidend, um mit der Analyse, Datenextraktion und Visualisierung fortzufahren.

In [31]:
# Legen Sie den ausgewählten Zeitraum für die Analyse fest (0: Jahr, 1: Sommer, 2: Winter)
selected_period = period_list[1]  # Ändern Sie diesen Index, um den Zeitraum zu ändern
print(f"Ausgewählter Zeitraum: {selected_period}")

# Definieren Sie den Pfad des extrahierten Ordners basierend auf dem ausgewählten Zeitraum
extract_folder = os.path.join(working_folder, selected_period)

Ausgewählter Zeitraum: summer


### 3.1 Erstellen eines DataFrame mit verfügbaren netCDF-Dateien

In [32]:
import re
import pandas as pd
import netCDF4 as nc
import numpy as np

def meta(filename):
    # Überprüfen, ob der Dateiname dem erwarteten Muster entspricht
    match = re.search(
        r"(?P<ds_statistic>mean|p10|p90)_(?P<ds_variable>Tmean|Tmax|Tmin)_(?P<ds_period>Yearly|Winter|Summer)_(?P<rcp>rcp\d{2})_(?P<rcp_statistic>mean|stdev)_v(?P<version>\d+\.\d+)\.",
        filename
    )
    
    # Fehler ausgeben, wenn der Dateiname nicht dem erwarteten Schema entspricht
    if not match:
        raise ValueError("Der angegebene Dateiname entspricht nicht dem erwarteten Namensschema")
    
    # Funktion zum Extrahieren des Variablennamens aus der NetCDF-Datei
    filepath = os.path.join(extract_folder, filename)
    def get_nc_variable():
        with nc.Dataset(filepath, 'r') as nc_dataset:
            variable_names = list(nc_dataset.variables.keys())

            primary_variable_index = 0  # Diesen Index bei Bedarf anpassen
            if primary_variable_index >= len(variable_names):
                raise IndexError("Der Index der Hauptvariable liegt außerhalb des gültigen Bereichs für die NetCDF-Variablen.")
            
            primary_variable = variable_names[primary_variable_index]
            primary_variable_shape = np.shape(nc_dataset[primary_variable])

            return primary_variable, primary_variable_shape
    
    # Metadaten als Dictionary zurückgeben
    return dict(
        filename=filename,
        path=os.path.join(extract_folder, filename),
        ds_period=match.group('ds_period'),
        ds_variable=match.group('ds_variable'),
        ds_statistic=match.group('ds_statistic'),
        variable_name=get_nc_variable()[0],
        variable_shape=get_nc_variable()[1],
        rcp=match.group('rcp'),
        rcp_statistic=match.group('rcp_statistic'),
    )

# Metadaten für alle NetCDF-Dateien im Ordner extrahieren
# Das Dictionary 'nc_files' enthält alle relevanten Metadaten der verfügbaren NetCDF4-Dateien
# Dieses Dictionary wird später verwendet, um die Dateien in GeoTiff zu konvertieren
nc_files = [meta(f) for f in os.listdir(extract_folder) if f.endswith('.nc')]
nc_files = sorted(nc_files, key=lambda x: (x['ds_variable'], x['ds_statistic']))

df_nc_files = pd.DataFrame.from_dict(nc_files)

# Pandas-Anzeigeoptionen anpassen
pd.options.display.max_colwidth = 30

# DataFrame anzeigen, ohne die Spalte 'path' darzustellen
df_nc_files.loc[:, df_nc_files.columns != 'path']


,filename,ds_period,ds_variable,ds_statistic,variable_name,variable_shape,rcp,rcp_statistic
0,mean_Tmax_Summer_rcp45_mea...,Summer,Tmax,mean,mean_Tmax_Summer,"(100, 82, 95)",rcp45,mean
1,mean_Tmax_Summer_rcp45_std...,Summer,Tmax,mean,mean_Tmax_Summer,"(100, 82, 95)",rcp45,stdev
2,mean_Tmax_Summer_rcp85_mea...,Summer,Tmax,mean,mean_Tmax_Summer,"(100, 82, 95)",rcp85,mean
3,mean_Tmax_Summer_rcp85_std...,Summer,Tmax,mean,mean_Tmax_Summer,"(100, 82, 95)",rcp85,stdev
4,mean_Tmean_Summer_rcp45_me...,Summer,Tmean,mean,mean_Tmean_Summer,"(100, 82, 95)",rcp45,mean
5,mean_Tmean_Summer_rcp45_st...,Summer,Tmean,mean,mean_Tmean_Summer,"(100, 82, 95)",rcp45,stdev
6,mean_Tmean_Summer_rcp85_me...,Summer,Tmean,mean,mean_Tmean_Summer,"(100, 82, 95)",rcp85,mean
7,mean_Tmean_Summer_rcp85_st...,Summer,Tmean,mean,mean_Tmean_Summer,"(100, 82, 95)",rcp85,stdev
8,p10_Tmean_Summer_rcp45_mea...,Summer,Tmean,p10,p10_Tmean_Summer,"(100, 82, 95)",rcp45,mean
9,p10_Tmean_Summer_rcp45_std...,Summer,Tmean,p10,p10_Tmean_Summer,"(100, 82, 95)",rcp45,stdev


### 3.2 Übersicht über die Struktur der heruntergeladenen Daten  

Das folgende Diagramm veranschaulicht die Struktur der **SIS-Temperaturstatistik-Daten**, die in vier hierarchische Ebenen unterteilt sind:  

1. **Zeitraum**: Gibt die Zeiteinheit der Daten an, die eine der folgenden sein kann:  
    - **Jahr**  
    - **Sommer**  
    - **Winter**  

2. **Variable**: Die Daten enthalten drei Temperaturvariablen:  
    - **Durchschnittstemperatur**  
    - **Mindesttemperatur**  
    - **Maximaltemperatur**  

3. **Statistik**: Für jede Variable werden folgende Statistiken berechnet:  
    - **Zeitlicher Durchschnitt**  
    - **10. Perzentil** (nur für Durchschnittstemperatur)  
    - **90. Perzentil** (nur für Durchschnittstemperatur)  

4. **Experiment**: Die Daten werden unter zwei Klimaszenarien bereitgestellt:  
    - **rcp45**  
    - **rcp85**  
    - Für jedes Szenario liegen die Daten in zwei Formen vor:  
        - **rcp45_mean**, **rcp45_std**  
        - **rcp85_mean**, **rcp85_std**  

Die Datenstruktur folgt einer hierarchischen Abfolge von **Zeitraum** zu **Variable**, dann zu **Statistik** und schließlich zu **Experiment**. Diese Hierarchie zeigt, wie die Temperaturdaten organisiert, berechnet und analysiert werden.  

<img src="./images/sis-temperature-statistics-data-structure_v2.jpg" width="550" style="display: block; margin: 0 auto; border: 1px solid #aeaeae">


### 3.3 Einzigartige Variablennamen und verfügbare Variablen ausgeben

In [33]:
# Variable definieren, um bereits verarbeitete Variablennamen zu speichern und Duplikate zu vermeiden  
seen_variables = set()  

# Alle Variablen in jeder NetCDF-Datei auflisten  
for i, nc_file in enumerate(nc_files):  
    variable_name = nc_file['variable_name']  
    
    # Überspringen, wenn die Variable bereits verarbeitet wurde  
    if variable_name in seen_variables:  
        continue  

    # NetCDF-Datei im Lesemodus öffnen  
    with nc.Dataset(nc_file['path'], mode='r') as nc_dataset:  
        # Alle Variablen im aktuellen Datensatz auflisten  
        variables_list = list(nc_dataset.variables.keys())  
        
        # Details der Datei und ihrer Variablen ausgeben  
        print(f"{i + 1:<2} {variable_name:<18}: Verfügbare Variablen: {variables_list}")  
    
    # Diese Variable als verarbeitet markieren  
    seen_variables.add(variable_name)  

# Variable definieren, um bereits verarbeitete Variablennamen zu speichern und Duplikate zu vermeiden  
seen_variables = set()  

1  mean_Tmax_Summer  : Verfügbare Variablen: ['mean_Tmax_Summer', 'height', 'lat', 'lon', 'time']
5  mean_Tmean_Summer : Verfügbare Variablen: ['mean_Tmean_Summer', 'height', 'lat', 'lon', 'time']
9  p10_Tmean_Summer  : Verfügbare Variablen: ['p10_Tmean_Summer', 'quantile', 'lat', 'lon', 'time']
13 p90_Tmean_Summer  : Verfügbare Variablen: ['p90_Tmean_Summer', 'quantile', 'lat', 'lon', 'time']
17 mean_Tmin_Summer  : Verfügbare Variablen: ['mean_Tmin_Summer', 'height', 'lat', 'lon', 'time']


In [34]:
# Alle Variableninformationen in jeder NetCDF-Datei auflisten  
for i, nc_file in enumerate(nc_files):  
    variable_name = nc_file['variable_name']  
    
    # Überspringen, wenn die Variable bereits verarbeitet wurde  
    if variable_name in seen_variables:  
        continue  
    
    # NetCDF-Datei im Lesemodus öffnen  
    with nc.Dataset(nc_file['path'], mode='r') as nc_dataset:  
        # Primärvariable-Daten abrufen  
        variable_data = nc_dataset[variable_name]  
        
        # Zusammenfassung der Primärvariable erstellen  
        summary = {  
            "Variablenname": variable_name,  
            "Datentyp": variable_data.dtype,  
            "Form": variable_data.shape,  
            "Variableninfo": f"{variable_data.dimensions}",  
            "Einheiten": getattr(variable_data, "units", "N/A"),  
            "Langer Name": getattr(variable_data, "long_name", "N/A"),  
        }  
        
        # Datensatz-Zusammenfassung als DataFrame zur besseren Visualisierung anzeigen  
        nc_summary = pd.DataFrame(list(summary.items()), columns=['Beschreibung', 'Bemerkungen'])  
        print(f"{i + 1}. Zusammenfassung der Variable '{variable_name}':")  
        display(nc_summary)  
    
    # Variablenname zur Liste der bereits verarbeiteten Variablen hinzufügen  
    seen_variables.add(variable_name)  
    
    # Ausgabe begrenzen  
    output_limit = 2  
    if len(seen_variables) >= output_limit:  
        print(f".... (Ausgabe auf die ersten {output_limit} Variablen gekürzt)")  
        break  


1. Zusammenfassung der Variable 'mean_Tmax_Summer':


,Beschreibung,Bemerkungen
0,Variablenname,mean_Tmax_Summer
1,Datentyp,float32
2,Form,"(100, 82, 95)"
3,Variableninfo,"('time', 'lat', 'lon')"
4,Einheiten,degrees C
5,Langer Name,Ensemble members average o...


5. Zusammenfassung der Variable 'mean_Tmean_Summer':


,Beschreibung,Bemerkungen
0,Variablenname,mean_Tmean_Summer
1,Datentyp,float32
2,Form,"(100, 82, 95)"
3,Variableninfo,"('time', 'lat', 'lon')"
4,Einheiten,degrees C
5,Langer Name,Ensemble members average o...


.... (Ausgabe auf die ersten 2 Variablen gekürzt)


## 4. Exportieren der NetCDF4-Dateien im csv-Format

In [2]:
## Dummy text

## 5. Exportieren der NetCDF4-Datei nach GeoTIFF

In [3]:
## Dummy text

## 6. Analyse und Visualisierung Optionen